In [17]:
import glob
import os
import re
import datetime

def get_spent_amount(text):
    spent_amt = []
    amt = 0.0
    debarred = ['|', '~—', '_', '‘Total', '“Total', '(Rs.)']
    for line in text:
        if 'total amount after tax' in line.lower():
            amt = line.split()[-1]
            if not amt[0].isalpha() and amt not in debarred:
                # print('yess1')
                spent_amt.append(amt)
        elif 'grand total' in line.lower() and not spent_amt:
            amt = line.split()[-1]
            if not amt[0].isalpha() and amt not in debarred:
                # print('yess2')
                spent_amt.append(amt)
        elif 'total' in line.lower() and not spent_amt:
            amt = line.split()[-1]
            if not amt[0].isalpha() and amt not in debarred:
                # print('yess3')
                spent_amt.append(amt)
        elif 'grand total with gst' in line.lower() and not spent_amt:
            amt = line.split()[-1]
            if not amt[0].isalpha() and amt not in debarred:
                # print('yess4')
                spent_amt.append(amt)

    if not spent_amt:
        spent_amt = [None]
    return spent_amt

def get_currency_code(text):
    all_text = ' '.join(t for t in text)
    keys = ['inr', 'rupees', 'rs']
    for key in keys:
        if key in all_text.lower():
            return ['INR']

def get_spent_date(text):
    all_text = ' '.join(t for t in text)
    flag = 0
    inv_date = []
    date = ''
    debarred = ['date.', '[dated', 'no.', '1dated', 'date,', 'due.', '‘dated', '|dated']

    for i in range(len(text)):

        if 'invoice date' in text[i].lower():
            date = text[i].split()[-1]
            # inv_date.append(date)
            if date.lower() not in debarred and ':' not in date and not date.isalpha() and len(date) > 4:
                inv_date.append(date)
                # print('DATE---> ', inv_date)
            break
        elif 'date' in text[i].lower() and not date:
            date = text[i].split()[-1]
            # print(date)
            # inv_date.append(date)
            if date.lower() not in debarred and ':' not in date and not date.isalpha() and len(date) > 4:
                inv_date.append(date)
                # print('DATE---> ', inv_date)
            break
        elif re.search('.*invoice no.*dated.*', text[i].lower()) and not date:
            date = text[i+1].split()[-1]
            # inv_date.append(date)
            if date.lower() not in debarred and ':' not in date and not date.isalpha() and len(date) > 4:
                inv_date.append(date)
                # print('DATE---> ', inv_date)
            break
        elif re.search('.*inv.*date.*', text[i].lower()):
            date = text[i].split()[-1]
            # inv_date.append(date)
            if date.lower() not in debarred and ':' not in date and not date.isalpha() and len(date) > 4:
                inv_date.append(date)
                # print('DATE---> ', inv_date)
            break
 
    inv_date_new = []
    for d in inv_date:
        formats = ['%d-%b-%Y', '%d-%m-%Y', '%d/%m/%Y', '%d-%b-%y']
        for format in formats:
                    try:
                        d = datetime.datetime.strptime(d, format).strftime('%Y-%m-%d')
                        inv_date_new.append(d)
                    except:
                        pass
    
    if not inv_date_new:
        inv_date_new = [None]
    return inv_date_new

def clean_name(s_name):
    words = s_name.split()
    new_words = s_name.lower().split()
    keys = ['invoice', 'lnyelee-no-']
    ind = 0

    for key in keys:
        if key in new_words:
            ind = new_words.index(key)
            break
    
    if ind:
        s_name = ' '.join(words[:ind])
    
    # if not s_name:
    #     s_name = [None]
    return s_name

def get_supplier_name(text):
    supp_name = []
    debarred = ['~', 'tax', 'ae', 'irn', 'aren', 'original', 'duplicate', 'transporter', 'si', '36dqypr6397n1z8', 'input', 'gst', 'ack', 'status', 'e-invoice']

    for i in range(len(text)):
        s_name = text[i]
        flag = 0
        for deb in debarred:
            if deb in s_name.lower():
                flag = 1
                break
        if flag == 1:
            continue
        if flag == 0:
            if re.search('.*[0-9].*', s_name):
                continue
            s_name = clean_name(s_name)
            # for s in s_name:
            #     if s.isnumeric():
            #         print(s_name)
            supp_name.append(s_name)
        break

    # if not supp_name:
    #     supp_name = [None]
    return supp_name

def get_stationary_asset(text):
    asset = []

    data = {
        'first': 'Dendukuri House',
        'second': 'Galaxy Aurobindo 9th Floor',
        'third': 'Galaxy Aurobindo 5th Floor',
    }

    for line in text:
        if re.search('.*plot no.*564/a39.*phase.*road no 92.*', line.lower()):
            asset.append(data['first'])
            break
        elif re.search('.*plot no.*564/a.*39.*', line.lower()):
            asset.append(data['first'])
            break
        elif re.search('.*plot.*564/a.*39.*', line.lower()):
            asset.append(data['first'])
            break
        elif re.search('.*phase.*road no. 92.*plot no.*', line.lower()):
            asset.append(data['first'])
            break
        elif re.search('.*9th.*aurobindo galaxy.*', line.lower()):
            # print('ASSET----> ', data['second'])
            asset.append(data['second'])
            break
        elif re.search('.*9th.*aurabindo.*', line.lower()):
            # print('ASSET----> ', data['second'])
            asset.append(data['second'])
            break
        elif re.search('.*aurobindo.*9th.*', line.lower()):
            # print('ASSET----> ', data['second'])
            asset.append(data['second'])
            break

    # if not asset:
    #     asset = [None]
    return asset

def clean_text(text):
    new_text = []
    for line in text:
        line = line.replace('|', '')
        line = line.replace('\\', '')
        line = line.replace(')', '')
        line = line.replace('(', '')
        line = line.replace('{', '')
        new_text.append(line)
    
    # if not new_text:
    #     new_text = [None]
    return new_text

def contains_float(line):
    words = line.split()
    for word in words:
        try:
            float(word)
            return True
        except ValueError:
            return False

def clean_item_names(item_names):
    # print('ITEM NAMES TO BE CLEANED---> ', item_names)
    debarred = ['no’s', 'nos', 'no', 'sno\'s', 'c19 out']
    for i in range(len(item_names)):
        if item_names[i].lower() in debarred:
            return item_names[:i]
        if contains_float(item_names[i]):
            return item_names[:i]

    ind = 0
    for j in range(len(item_names)):
        words = item_names[j].split()
        for i in range(len(words)):
            for deb in debarred:
                if deb in words[i].lower():
                    ind = i
                    break
            if ind:
                break
        if ind:
            item_names[j] = ' '.join(words[:ind])
        # return item_names

    debarred2 = ['sees!', 'so ae sages', 'eas!', ': —', 'nos', 'no\'s']
    new_item_names = []
    remove_items = []

    # print('ITEM NAMES---> ', item_names)

    for deb in debarred2:
        for item in item_names:
            # print('ITEM---> ', item)
            if deb in item.lower():
                # print('ITEM---> ', item)
                remove_items.append(item)

    # print('REMOVED ITEMS---> ', remove_items)
    new_item_names = [item for item in item_names if item not in remove_items]

    if new_item_names:
        return new_item_names
        
    return item_names

def get_item_names(text):

    debarred = ['services', 'tax', 'eee', 'sens', 'aia', 'serial', 'eas!', 'sa rs ie.', '[eels', 'noï¿½s']
    item_names = []
    text = clean_text(text)

    for i in range(len(text)):
        if ('particulars' in text[i].lower() and 'described' not in text[i].lower()) or 'description' in text[i].lower() or 'quantity' in text[i].lower():
            for j in range(i+1, len(text)):
                if re.search('.*total.*', text[j].lower()) or re.search('.*totai.*', text[j].lower()):
                    break
                flag = 0
                for deb in debarred:
                    if deb in text[j].lower():
                        flag = 1
                        break
                if flag == 1:
                    continue
                words = text[j].split()
                name = ''
                for k in range(len(words)):
                    if k == 0:
                        if words[k].isnumeric():
                            # flag2 = 1
                            continue
                    if words[k].isnumeric() and len(words[k]) > 3:
                        break
                    else:
                        # if flag2 == 1:
                        if re.search('.*no.*s', text[j].lower()) and 'total' not in text[j].lower():
                            name += words[k] + " "
                        elif contains_float(text[j]):
                            name += words[k] + " "
                        
                if name:
                    item_names.append(name.strip())
    
    # print('ITEM NAMES---> ', item_names)
    item_names = clean_item_names(item_names)
    # print('ITEM NAMES---> ', item_names)
    # if not item_names:
    #     item_names = [None]
    return item_names

def get_quantity(text):
    # item_names = get_item_names(text)
    text = clean_text(text)
    quantities = []
    q_line = []
    flag = 0

    for i in range(len(text)):
        if 'quantity' in text[i].lower():
            for j in range(i+1, len(text)):
                if 'total' in text[j].lower() or 'signature valid' in text[j].lower() or 'output cgst' in text[j].lower():
                    flag = 1
                    break
                if 'services' in text[j].lower() or 'serial' in text[j].lower():
                    continue
                if 'no. services' in text[j].lower():
                    continue
                if re.search('.*no.*s.*', text[j].lower()) or re.search('.*no.*', text[j].lower()):
                    # print('TRUUEEE')
                    q_line.append(text[j])
        if flag:
            break
    
    debarred = ['=']

    # print('QUANTITY LINE---> ', q_line)
    for q in q_line:
        words = q.split()
        for i in range(len(words)):
            if re.search('.*no.*s.*', words[i].lower()) or re.search('.*no.*', words[i].lower()):
                if any(char.isdigit() for char in words[i].lower()) and words[i].lower() not in debarred and words[i][0].isnumeric():
                    quantities.append(words[i])
                else:
                    if words[i-1].lower() not in debarred:
                        quantities.append(words[i-1])
                break
    
    # print('QUANTITIES---> ', quantities)
    new_quantities = []
    # flag = 0
    for quan in quantities:
        new_quan = ''
        if 'sno\'s' in quan.lower():
            break
        for ch in quan:
            if ch.isnumeric() or ch == '.':
                new_quan += ch
            else:
                flag = 1
                break
        if (new_quan.isnumeric() or contains_float(new_quan)) and len(new_quan) < 8:
            new_quantities.append(new_quan)

    # if not new_quantities:
    #     new_quantities = [None]
    return new_quantities

def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

def get_line_item_amount(text):
    line_item_amounts = []
    flag = 0

    for i in range(len(text)):
        if re.search('.*quantity.*amount.*', text[i].lower()):
            for j in range(i+1, len(text)):
                if 'igst' in text[j].lower():
                    flag = 1
                    break
                # print(text[j].split()[-1])
                # print('CONTAINS FLOAT---> ', isfloat(text[j].split()[-1]))
                amt = text[j].split()[-1].replace(',', '')
                # print('AMT---> ', amt)
                if len(text[j].split()) > 1 and isfloat(amt):
                    # print('AMOUNT---> ', text[j].split()[-1])
                    line_item_amounts.append(text[j].split()[-1])
        if flag:
            break
    
    return line_item_amounts

stationary_asset_sheet = []
spent_amount_sheet = []
currency_code_sheet = []
spent_date_sheet = []
supplier_name_sheet = []
item_names_sheet = []
quantities_sheet = []
line_item_amounts_sheet = []
files = []

for file in glob.glob('C:\\air_ticket\\supplier_comakeit\\ensemble_files1_new\\4_DKM*.txt'):
    print(file)
    files.append(file)
    filename = os.path.split(file)[-1]

    stationary_asset_final = []
    spent_amount_final = []
    currency_code_final = []
    spent_date_final = []
    supplier_name_final = []
    item_names_final = []
    quantities_final = []
    line_item_amounts_final = []
    
    with open(file) as fp:

        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts:
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            # pprint(text)
            filename = os.path.split(file)[-1].split('.txt')[0]

            spent_amount = get_spent_amount(text)
            if spent_amount and not spent_amount_final:
                spent_amount_final = spent_amount

            currency_code = get_currency_code(text)
            if currency_code and not currency_code_final:
                currency_code_final = currency_code

            spent_date = get_spent_date(text)
            if spent_date and not spent_date_final:
                spent_date_final = spent_date

            supplier_name = get_supplier_name(text)
            if supplier_name and not supplier_name_final:
                supplier_name_final = supplier_name

            stationary_asset = get_stationary_asset(text)
            if stationary_asset and not stationary_asset_final:
                stationary_asset_final = stationary_asset

            item_names = get_item_names(text)
            if item_names and not item_names_final:
                item_names_final = item_names
            if len(item_names) > len(item_names_final):
                item_names_final = item_names

            quantities = get_quantity(text)
            if quantities and not quantities_final:
                quantities_final = quantities
            if len(quantities) > len(quantities_final):
                quantities_final = quantities

            line_item_amounts = get_line_item_amount(text)
            if line_item_amounts and not line_item_amounts_final:
                line_item_amounts_final = line_item_amounts

    if not item_names_final:
        item_names_final = [None]
    item_names_sheet.extend(item_names_final)
  
    if not stationary_asset_final:
        stationary_asset_final = [None]
    stationary_asset_sheet.extend(stationary_asset_final*len(item_names_final))

    if not spent_amount_final:
        spent_amount_final = [None]
    spent_amount_sheet.extend(spent_amount_final*len(item_names_final))

    if not currency_code_final:
        currency_code_final = [None]
    currency_code_sheet.extend(currency_code_final*len(item_names_final))

    if not spent_date_final:
        spent_date_final = [None]
    spent_date_sheet.extend(spent_date_final*len(item_names_final))

    if not supplier_name_final:
        supplier_name_final = [None]
    supplier_name_sheet.extend(supplier_name_final*len(item_names_final))

    if not quantities_final:
        quantities_final = [None]
    l = 0
    if len(quantities_final) < len(item_names_final):
        l = len(item_names_final) - len(quantities_final)
    if len(item_names_final) < len(quantities_final):
        l = len(quantities_final) - len(item_names_final)
    if l:
        quantities_final.extend([None]*l)
    quantities_sheet.extend(quantities_final)
    # quantities_sheet.extend([None]*l)

    # if not item_names_final:
    #     item_names_final = [None]
    # item_names_sheet.extend(item_names_final)
    
    print(stationary_asset_final)
    print(spent_amount_final)  
    print(currency_code_final) 
    print(spent_date_final)
    print(supplier_name_final)
    print(item_names_final)
    print(quantities_final)
    print(line_item_amounts_final)
    print('------------------------------------') 
    
# # print(spent_date_sheet)
# print(len(stationary_asset_sheet))
# print(len(spent_amount_sheet))
# print(len(currency_code_sheet)) 
# print(len(spent_date_sheet))
# print(len(supplier_name_sheet))
# print(len(item_names_sheet))
# print(len(quantities_sheet))
# # print(len(files))
# print('------------------------------------')      

C:\air_ticket\supplier_comakeit\ensemble_files1_new\4_Dkm_14972022-2023_09.07.2022_page0.txt
['Dendukuri House']
['29,601.00']
['INR']
['2022-07-09']
['DKM ONLINE PRIVATE LIMITED']
['Fee for PF, Payroll & ESI', 'Fee for PF, Payroll & ESI']
[None, None]
['13,727.00', '11,359.00']
------------------------------------


In [153]:
import pandas as pd
df = pd.DataFrame({
    'Stationary Asset': stationary_asset_sheet,
    'Item Name': item_names_sheet,
    'Spent Amount': spent_amount_sheet,
    'Currency Code': currency_code_sheet,
    'Quantity': quantities_sheet,
    'Spent Date': spent_date_sheet,
    'Supplier Name': supplier_name_sheet
})

df.to_excel('supplier_comakeit_data.xlsx')